In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch

In [4]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [15]:
df = pd.read_csv('./data/fashion-mnist_train.csv')
df.shape

(60000, 785)

In [6]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X = df.drop(columns=['label']).values
y = df['label'].values

In [17]:
X.shape,y.shape

((60000, 784), (60000,))

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [20]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [21]:
X_train.shape

(48000, 784)

In [22]:
class CustomDataset(Dataset):
    def __init__(self,features,labels):
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [23]:
train_dataset = CustomDataset(X_train,y_train)

In [24]:
test_dataset = CustomDataset(X_test,y_test)

In [25]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_channels):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            
            nn.Linear(64,10)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        
        return x

In [28]:
learning_rate = 0.01
epochs = 100

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
model = CNN(1)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [38]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

c:\Users\Ojas\.vscode\Repos\Deep-Learning\venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch: 1 , Loss: 0.6686663827697436
Epoch: 2 , Loss: 0.3911582736596465
Epoch: 3 , Loss: 0.324316957761844
Epoch: 4 , Loss: 0.2955721357402702
Epoch: 5 , Loss: 0.26635316174725693
Epoch: 6 , Loss: 0.2471483848138402
Epoch: 7 , Loss: 0.2311958857489129
Epoch: 8 , Loss: 0.21700239385167758
Epoch: 9 , Loss: 0.19896126747503876
Epoch: 10 , Loss: 0.18670138462260366
Epoch: 11 , Loss: 0.17897397918999194
Epoch: 12 , Loss: 0.1694671458232527
Epoch: 13 , Loss: 0.16032170066982507
Epoch: 14 , Loss: 0.15341865917450437
Epoch: 15 , Loss: 0.14125795895109575
Epoch: 16 , Loss: 0.1363020295308282
Epoch: 17 , Loss: 0.1280665086157387
Epoch: 18 , Loss: 0.1183270439041468
Epoch: 19 , Loss: 0.11369498602347448
Epoch: 20 , Loss: 0.11428164392787342
Epoch: 21 , Loss: 0.107937651900885
Epoch: 22 , Loss: 0.10036978245914603
Epoch: 23 , Loss: 0.09748206684078711
Epoch: 24 , Loss: 0.09050140879761118
Epoch: 25 , Loss: 0.08629707620774085
Epoch: 26 , Loss: 0.0819339408527594
Epoch: 27 , Loss: 0.079192868472348

KeyboardInterrupt: 